# use polynomial function as integrand and normal distribution as probability measure

## different type of control variates are implemented

In [23]:
## load modules
from scipy import spatial
#import matplotlib.pyplot as plt
import numpy as np
# load modules
import torch
from scipy import stats
#%matplotlib inline
from scipy import special
import time,pdb,json
from copy import copy
from collections import defaultdict
from torch.autograd import Variable
from torch import nn
from torch.autograd import grad
import torch.nn.functional as F
from scipy.spatial.distance import cdist
from sklearn.metrics.pairwise import euclidean_distances
### import control variates from other files
from neural_networks_control_variates import SteinFirstOrderNeuralNets
from GaussianKernelControlVariates import GaussianControlVariate,OatesGram
from polynomial_control_variates import SteinSecondOrderQuadPolyCV


### integrand:  f(x1, x2,...,x_m) = \Sum_{i=1}^{m}\Sum_{j=0}^{n-1}Alpha_{ij}x_i^{j}
### normal probability measure: (x1, x2, ..., x_m) \sim N(0, sigma^2)

In [24]:
# compute the integration and the gradient
class Prepare_Poly(object):
    def __init__(self, Alpha, sigma):
        """
        initialization of integrands
        f(x1, x2,...,x_m) = \Sum_{i=1}^{m}\Sum_{j=0}^{n-1}Alpha_{ij}x_i^{j}

        :params[in], Alpha, two-dimensional torch array, i.e., matrix
        :params[in], sigma, the standard deviation of Gaussian distr.
        """
        self.Alpha = Alpha  # matrix of constants in the integrand
        self.dim, self.order = Alpha.size() # dimension of x, and the highest order of polynomial
        self.sigma = sigma
        
    def vect_power_matrix(self, x):
        """
        for each input 1-d tensor x, find the matrix of its powers
        
        :params[in], x, a d-dimensional 1-d tensor
        
        :params[out], res, a 2-d tensor, each column is x^i
        """
        powers = [x.pow(i) for i in range(self.order)]
        res = torch.stack(powers, dim=1) # dim * order 2-d tensor
        return res
        
    def integrand(self, x):
        """
        evaluate the integrand function
        
        :params[in], x, a one-dimensional tensor 
        
        :params[out], res, a real number
        """
        pow_mat = self.vect_power_matrix(x) # power matrix
        hadam = self.Alpha * pow_mat  # hadamard product
        res = torch.sum(hadam, dim=1).sum().item() # evaluation
        return res
    
    def integral(self):
        """
        compute the real value of the integral
        
        :params[out], res, a real number
        """
         # values of right part of Eq. 10
        vals = [(self.sigma**i)*special.factorial2(i-1).item() if i%2==0 \
                else 0 for i in range(0, self.order)]
        prod_mat_vec = torch.matmul(self.Alpha, torch.tensor(vals))
        res = prod_mat_vec.sum().item()
        return res
    
    def score_func(self, x):
        """
        evaluate the score function, i.e., gradient of log density
        
        :params[in], x, a one-dimensional tensor 
        
        :params[out], score, a 1-d tensor
        """
        score = -1*x/(self.sigma**2)
        return score
    
    def sample(self, n):
        """
        draw a sample from the multivariate Gaussian distr.
        
        :params[in], n, an integer, sample size
        
        :params[out], samp, a n-by-d matrix tensor, of which each row represents a sample
        """
        samp = self.sigma*torch.randn(n, self.dim)
        return samp



In [12]:
X.size()

torch.Size([10000, 2])

In [17]:
np.zeros(5)

array([0., 0., 0., 0., 0.])

### high-dimensional polynomial integrand with respect to normal distribution

In [30]:
dim = 10
t1 = [[1., -1.] for i in range(dim)]
Alpha, sigma = torch.tensor(t1), 1. # setup
prob1 = Prepare_Poly(Alpha, sigma)  # problem 1
D_in = Alpha.size()[0]              # dimension of x
# simulate data
size = 10000  # sample size
result_epochs = [3, 5, 10, 15, 20] # the epochs at which the results are recorded
num_epochs = len(result_epochs)
mc_err, cv_err, comp_time = np.zeros(num_epochs),np.zeros(num_epochs),\
    np.zeros(num_epochs)
cls_err,cls_full,cls_time = np.zeros(nrep),np.zeros(nrep),np.zeros(nrep)
train_perc, valid_perc = 0.9, .1
# for i in range(nrep):   ## repeated sampling
X = prob1.sample(size)    # draw sample
Y = torch.tensor([prob1.integrand(X[i]) for i in range(size)]) # f(x[i])
score_matrix = prob1.score_func(X)   ## score matrix for the data
integral = prob1.integral()          ## true value of integral

### Quadratic Second order Stein Control variates

In [27]:
lr, gamma = 2.e-3, .5       ## learning rate, gamma is the learning rate reducing rate
cv1 = SteinSecondOrderQuadPolyCV(D_in, score_matrix)       # Quadratic control variate
res = cv1.train(X, Y, integral, lr, gamma, clip_value=10.0, train_perc=.3, step_size=10,\
  weight_decay = 0.0, batch_size = 8, result_epochs=result_epochs, norm_init_std=1.e-4)
## monte carlo error, control variates error and computing time
mc_err, cv_err, comp_time = res[:3]
print('Monte Carlo Error:', mc_err, ' Control Variates Error:',cv_err, ' Computing Time:', comp_time)
## use classical method --- to find exact solution
res1 =cv1.classical_way(X, Y, integral, train_perc=train_perc)
cls_err, cls_time = res1[1:3]     ## exact solution and its computing time
print('Exact Control Variates Error:',cls_err,' Exact Control Variates Compute time:',cls_time)

0 -th Epoch, training loss:  1.163469209432602 current validation loss:  1.203547460079193 best val loss: 1.203547460079193
1 -th Epoch, training loss:  0.3103211639722188 current validation loss:  0.3297775376353945 best val loss: 0.3297775376353945
2 -th Epoch, training loss:  0.07677746645609537 current validation loss:  0.08308441940375737 best val loss: 0.08308441940375737
3 -th Epoch, training loss:  0.019481076876322428 current validation loss:  0.021141732164791652 best val loss: 0.021141732164791652
4 -th Epoch, training loss:  0.005055023511250814 current validation loss:  0.005539442896842956 best val loss: 0.005539442896842956
5 -th Epoch, training loss:  0.0013356804847717285 current validation loss:  0.0014703112500054495 best val loss: 0.0014703112500054495
6 -th Epoch, training loss:  0.0003575609525044759 current validation loss:  0.00039496185098375594 best val loss: 0.00039496185098375594
7 -th Epoch, training loss:  9.714746475219727e-05 current validation loss:  0.

### Kernel Control variate with the kernel defined in Oates et al. 2017 JRSSB paper

In [31]:
dim = 10
t1 = [[1., -1.] for i in range(dim)]
Alpha, sigma = torch.tensor(t1), 1. # setup
prob1 = Prepare_Poly(Alpha, sigma)  # problem 1
D_in = Alpha.size()[0]              # dimension of x
# simulate data
size = 1000  # sample size
result_epochs = [3, 5, 10, 15, 20] # the epochs at which the results are recorded
num_epochs = len(result_epochs)
mc_err, cv_err, comp_time = np.zeros(num_epochs),np.zeros(num_epochs),\
    np.zeros(num_epochs)
cls_err,cls_full,cls_time = np.zeros(nrep),np.zeros(nrep),np.zeros(nrep)
# for i in range(nrep):   ## repeated sampling
X = prob1.sample(size)    # draw sample
Y = torch.tensor([prob1.integrand(X[i]) for i in range(size)]) # f(x[i])
score_matrix = prob1.score_func(X)   ## score matrix for the data
integral = prob1.integral()          ## true value of integral

In [34]:
mean_distance = float(euclidean_distances(X).mean())    ## mean distance of all samples in X, each row represents a sample
lr, gamma = 2.e-3, .5             ## learning rate, gamma is the learning rate reducing rate
coef1, coef2 = 0.1, 1.0         ## coefficients to multiply mean_distance to get the parameters in Oates et al. Kernel;
kernel_alpha1, kernel_alpha2 = mean_distance*coef1, mean_distance*coef2
train_pct1 = .5       ## percentage of data used for training
result_epochs = [3, 5, 10, 15, 20] # the epochs at which the results are recorded
num_epochs = len(result_epochs)
mc_err, cv_err, comp_time = np.zeros(num_epochs),np.zeros(num_epochs),\
    np.zeros(num_epochs)
cls_err,cls_time = np.zeros(nrep),np.zeros(nrep)   ## least square Control variates error

## evaluate the gram matrix for the kernel
start_time = time.time()
ker2= OatesGram(X.data.numpy(), score_matrix.data.numpy(), alpha1=kernel_alpha1, alpha2=kernel_alpha2)
gram_matrix = ker2.gram_matrix()
gram_time = time.time()-start_time  ## computing time for gram matrix
sigma_sq = 0.0     ## sigma square to add on the gram matrix 
kernel_cv1 = GaussianControlVariate(X, Y, gram_matrix=gram_matrix, gram_time=gram_time, sigma_sq=sigma_sq,\
                                    train_pct=train_pct1)
ls_res1=kernel_cv1.classical_way(Y, integral)    ## use least square to find exact solution
cv_res1=kernel_cv1.train(Y, integral, lr, gamma, clip_value=dim/10., valid_perc=(1.-train_pct1),\
                         batch_size=4, N_epochs=result_epochs, step_size=10, weight_decay=1.0e-3)
## classical least square to train control variates
mc_err, cls_err, cls_time = ls_res1[:3]  ## monte carlo error, exact solution and its computing time
##  control variates result --- control variates error and computing time
cv_err, comp_time = cv_res1[:2]
print('Monte Carlo Error:', mc_err, ' Control Variates Error:',cv_err, ' Computing Time:', comp_time)
## use classical method --- to find exact solution
print('Exact Control Variates Error:',cls_err,' Exact Control Variates Compute time:',cls_time)

0 -th Epoch, training loss:  2.4137000312805177 current validation loss:  2.3043442754745485 best val loss: 2.3043442754745485
1 -th Epoch, training loss:  2.330137397766113 current validation loss:  2.2285582246780398 best val loss: 2.2285582246780398
2 -th Epoch, training loss:  2.2449100580215453 current validation loss:  2.1509262647628784 best val loss: 2.1509262647628784
3 -th Epoch, training loss:  2.1612113466262817 current validation loss:  2.075430911064148 best val loss: 2.075430911064148
4 -th Epoch, training loss:  2.077588797569275 current validation loss:  2.000262734413147 best val loss: 2.000262734413147
5 -th Epoch, training loss:  1.993915514945984 current validation loss:  1.9242565221786498 best val loss: 1.9242565221786498
6 -th Epoch, training loss:  1.9143200874328614 current validation loss:  1.852272801399231 best val loss: 1.852272801399231
7 -th Epoch, training loss:  1.835290244102478 current validation loss:  1.7811649522781372 best val loss: 1.78116495227

#### comments: the performance of kernel control variates is very sensitive to the choice of hyper-parameters (kernel_alpha1, kernel_alpha2, determined by coef1, and coef2)

### Neural Networks Control variates 

In [36]:
dim = 10
t1 = [[1., -1.] for i in range(dim)]
Alpha, sigma = torch.tensor(t1), 1. # setup
prob1 = Prepare_Poly(Alpha, sigma)  # problem 1
D_in = Alpha.size()[0]              # dimension of x, i.e., input
# simulate data
size = 1000  # sample size
result_epochs = [3, 5, 10, 15, 20] # the epochs at which the results are recorded
num_epochs = len(result_epochs)
cv_err, comp_time = np.zeros(num_epochs),np.zeros(num_epochs)

# for i in range(nrep):   ## repeated sampling
X = prob1.sample(size)    # draw sample
Y = torch.tensor([prob1.integrand(X[i]) for i in range(size)]) # f(x[i])
score_matrix = prob1.score_func(X)   ## score matrix for the data
integral = prob1.integral()          ## true value of integral

In [35]:

def count_parameters(model):
    """
    count the number of training parameters that require gradient
    
    :params[in]: model, a torch model that has parameters method
    
    :params[out]: total number of parameters
    """
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


In [41]:
drop_prob, lr, train_pct = 0.5, 1.e-4, .5    ## drop-out probability, learning rate, percent of data for training 
w_decay, gamma = 5.e-5,.5           ## weight decay, learning rate decay rate
## this is the input for neural networks control variates, requires gradients
nn_X = X.clone().detach().requires_grad_(True)
## control variates with neural nets -- h_dims is the number of neurons in each layer
nn_cv1 = SteinFirstOrderNeuralNets(D_in, h_dims=[20]*5, score_matrix=score_matrix,\
                                   init_val=Y.mean(), drop_prob=drop_prob)
## split the data into two parts, use Mean squared error to train
nn_res1=nn_cv1.trainer(nn_X, Y, integral, lr, gamma, clip_value=.1*count_parameters(nn_cv1),\
                       train_perc=train_pct, valid_perc=(1.-train_pct), batch_size =8,\
                       weight_decay = w_decay, result_epochs=result_epochs)
mc_err=Y.mean().item() - integral  ## monte carlo error
cv_err, comp_time = nn_res1[1:3]     ## control variates error/ and computing time over epochs


0 -th Epoch, training loss:  2.5397046327590944 validation loss:  2.5727998466491697 indicator: 0.000353968063776847
1 -th Epoch, training loss:  2.5194113245010374 validation loss:  2.5522395210266113 indicator: 0.000813267309218645
2 -th Epoch, training loss:  2.488999274253845 validation loss:  2.5214228725433347 indicator: 0.0014959034211933613
3 -th Epoch, training loss:  2.439742917060852 validation loss:  2.4715623817443846 indicator: 0.0025470267517957836
4 -th Epoch, training loss:  2.342993344306946 validation loss:  2.37383166885376 indicator: 0.004434479166287929
5 -th Epoch, training loss:  2.146729588508606 validation loss:  2.175960771560669 indicator: 0.007917857009917498
6 -th Epoch, training loss:  1.7766536855697632 validation loss:  1.8036571350097657 indicator: 0.01346542956493795
7 -th Epoch, training loss:  1.212803241252899 validation loss:  1.2357189159393311 indicator: 0.02138813330233097
8 -th Epoch, training loss:  0.7240641634464264 validation loss:  0.7425

In [42]:
mc_err

0.034076690673828125

In [43]:
cv_err

[0.05635274417325853,
 0.05341416842816393,
 0.02411524971574508,
 0.024459737822413175,
 0.02220653451234078]

#### comment: Tuning hyper-parameters like learning rate and weight decay rate can improve the performance